In [1]:
import time
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Importar dados

As labels dos dados estão definidas tais que:
- Background: 0
- Sinal: 1

In [18]:
# Gather data
bkgd = pd.read_hdf("Data/Background.h5", key='bkgd')
fcnc = pd.read_hdf("Data/FCNC.h5", key="fcnc")
data = pd.concat([bkgd, fcnc])
data

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,gen_weights,Label
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.482720,158.952774,595.341003,...,0.443487,407.632080,-0.755916,0.785445,292.118774,2.734722,2,1212.756226,1.304437e-07,0
1,-2.060421,30.932735,-1.365277,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,-2.106180,54.675926,2.572098,0.000000,0.000000,0.000000,1,352.939209,1.304437e-07,0
2,-1.025947,40.282574,-1.773086,0.288352,26.201660,-0.694144,2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,419.889740,1.304437e-07,0
3,1.084838,82.556099,2.932473,0.000000,0.000000,0.000000,1,0.969367,103.233589,318.344971,...,-0.059388,43.076065,-2.346566,0.000000,0.000000,0.000000,1,610.762146,1.304437e-07,0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,1.464516,80.244713,-2.605392,1.797688,23.799982,2.562366,2,291.874695,1.304437e-07,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,2.364251,79.450447,-2.115078,1.581713,46.098675,0.268987,2,306.712402,3.264224e-09,1
496618,-0.847799,214.928589,-2.360383,0.059395,31.239508,-2.978243,2,-0.959045,70.461861,257.986145,...,-1.299547,46.357815,-2.891009,0.000000,0.000000,0.000000,1,541.768921,3.264224e-09,1
496623,-0.230892,80.927383,-0.913889,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,-0.219239,120.608452,1.697549,0.000000,0.000000,0.000000,1,305.327148,3.264224e-09,1
496624,0.584297,143.974167,2.141792,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,-0.948717,27.718317,-0.361032,0.000000,0.000000,0.000000,1,404.741730,3.264224e-09,1


In [3]:
# Split data into training and testing
index_train, index_tests = train_test_split(data.index, test_size=2/3, random_state=42)
index_val, index_test = train_test_split(index_tests, test_size=0.5, random_state=40)
data.loc[index_train]

,Electron1_Eta,Electron1_PT,Electron1_Phi,Electron2_Eta,Electron2_PT,Electron2_Phi,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,...,Muon1_Eta,Muon1_PT,Muon1_Phi,Muon2_Eta,Muon2_PT,Muon2_Phi,Muon_Multi,ScalarHT_HT,gen_weights,Label
196887,-0.878250,283.007782,0.079671,0.780247,46.576385,1.320449,3,-0.878928,17.404884,285.370880,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,639.662415,3.264224e-09,1
472518,-0.303675,453.381500,-0.864273,0.103012,99.619553,-0.731962,2,-0.230816,97.797729,559.695923,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,806.408142,3.264224e-09,1
475171,1.209169,552.183472,-1.400386,0.929683,37.286667,-1.883680,2,1.198922,79.592354,585.456116,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1309.175293,3.264224e-09,1
437616,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.474208,105.589462,1.497597,1.405127,42.719090,1.311667,3,278.394287,3.264224e-09,1
20134,0.295409,82.163155,2.225906,0.282610,76.267296,-2.855317,2,0.000000,0.000000,0.000000,...,0.869774,35.877281,-0.300437,0.000000,0.000000,0.000000,1,255.104126,3.264224e-09,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303794,-1.662792,137.118622,-2.059103,-0.673272,45.620079,-2.562727,2,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,365.953430,3.264224e-09,1
262256,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,1.770717,95.972099,1.236606,1.268185,79.199051,-1.349634,3,276.898834,3.264224e-09,1
334458,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,-1.838968,23.493359,444.905212,...,-1.836683,441.275330,-2.321132,0.020551,19.692665,1.024954,2,896.008667,3.264224e-09,1
372614,-0.932552,78.131271,2.846272,-1.163761,62.170326,1.459902,2,-2.807124,31.390705,294.881683,...,-1.770924,126.805092,0.749870,0.000000,0.000000,0.000000,1,604.675842,3.264224e-09,1


In [4]:
print(f"Train_size: {len(index_train)} | Val_size: {len(index_val)} | Test_size: {len(index_test)}")

Train_size: 65241 | Val_size: 65242 | Test_size: 65242


In [5]:
# Get training features and labels
X_train = data.loc[index_train].drop(["Label", "gen_weights"], axis=1).values
y_train = data.loc[index_train]["Label"].values

# Get validation training features and labels
X_val = data.loc[index_val].drop(["Label", "gen_weights"], axis=1).values
y_val = data.loc[index_val]["Label"].values

# Get testing features and labels
X_test = data.loc[index_test].drop(["Label", "gen_weights"], axis=1).values
y_test = data.loc[index_test]["Label"].values

# Get sample weights
train_weights = data.loc[index_train]["gen_weights"].values
val_weights = data.loc[index_val]["gen_weights"].values
test_weights = data.loc[index_test]["gen_weights"].values

# Get class weights
class_weights = {
    "0": 1 - y_train[y_train == 0].sum() / len(y_train),
    "1": 1 - y_train[y_train == 1].sum() / len(y_train)
}

In [8]:
print(y_train)

[1 1 1 ... 1 1 1]


In [6]:
print(f"Class weights:\n{class_weights}\n")
print(f"Train_weights:\n{train_weights}\n")
print(f"Val_weights:\n{val_weights}\n")
print(f"Test_weights:\n{test_weights}\n")
print(f"X_train:\n{X_train}\n")
print(f"y_train:\n{y_train}\n")
print(f"X_val:\n{X_val}\n")
print(f"y_val:\n{y_val}\n")
print(f"X_test:\n{X_test}\n")
print(f"y_test:\n{y_test}\n")

Class weights:
{'0': 1.0, '1': 0.0}

Train_weights:
[3.26422429e-09 3.26422429e-09 3.26422429e-09 ... 3.26422429e-09
 3.26422429e-09 3.26422429e-09]

Val_weights:
[3.26422429e-09 3.26422429e-09 3.26422429e-09 ... 3.26422429e-09
 3.26422429e-09 3.26422429e-09]

Test_weights:
[3.26422429e-09 3.26422429e-09 3.26422429e-09 ... 3.26422429e-09
 3.26422429e-09 3.26422429e-09]

X_train:
[[-8.78249645e-01  2.83007782e+02  7.96714723e-02 ...  0.00000000e+00
   0.00000000e+00  6.39662415e+02]
 [-3.03674549e-01  4.53381500e+02 -8.64272535e-01 ...  0.00000000e+00
   0.00000000e+00  8.06408142e+02]
 [ 1.20916903e+00  5.52183472e+02 -1.40038598e+00 ...  0.00000000e+00
   0.00000000e+00  1.30917529e+03]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.02495360e+00
   2.00000000e+00  8.96008667e+02]
 [-9.32551861e-01  7.81312714e+01  2.84627247e+00 ...  0.00000000e+00
   1.00000000e+00  6.04675842e+02]
 [-2.06157193e-01  1.61139130e+02 -1.34389591e+00 ...  0.00000000e+00
   1.00000000e+00 

# Build the model

In [ ]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_samples):
        self.means = np.mean(data_samples, axis=0, keepdims=True)
        self.stds = np.std(data_samples, axis=0, keepdims=True)
        
    def call(self, inputs):
        return (inputs - self.means) / (self.stds + keras.backend.epsilon())
    
std_layer = Standardization()
std_layer.adapt(X_train)

In [ ]:
inputs = keras.Input(shape=(69,))
standardize = std_layer(inputs)
fc1 = keras.layers.Dense(90, activation='relu')(standardize)
fc2 = keras.layers.Dense(120, activation='relu')(fc1)
fc3 = keras.layers.Dense(80, activation='relu')(fc2)
fc4 = keras.layers.Dense(40, activation='relu')(fc3)
outputs = keras.layers.Dense(2, activation='softmax')(fc4)

model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss="sparse_categorical_crossentropy",
              metrics=["accuracy", keras.metrics.AUC()])
model.summary()

In [ ]:
# Model name
name = str(time.time())

# Callbacks 

In [ ]:
# Tensorboard
TB = keras.callbacks.TensorBoard("logs/" + name)

# Early Stopping
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, verbose=2, mode="min")

# Model Checkpoint
MC = keras.callbacks.ModelCheckpoint("models/" + name + ".h5", save_best_only=True, monitor="val_loss",
                                     mode="min")

# Treinar modelo

In [ ]:
history = model.fit(X_train, y_train, batch_size=1024, epochs=20, validation_data=(X_val, y_val),
                    sample_weight=train_weights, shuffle=True, callbacks=[TB, ES, MC])